# cuDF/Java with IJava kernel https://github.com/SpencerPark/IJava

Maven Install spark-rapids-jni or use a remote repo

In [9]:
%%loadFromPOM
<repository>
  <id>local-m2</id>
  <url>file:/home/gshegalov/.m2/repository</url>
</repository>
<dependency>
  <groupId>com.nvidia</groupId>
  <artifactId>spark-rapids-jni</artifactId>
  <version>23.10.0-SNAPSHOT</version>
  <classifier>cuda11</classifier>
</dependency>

In [10]:
import ai.rapids.cudf.*

In [ ]:
Table table0 = new Table.TestBuilder().column(1, 2, 3, 4).build()

In [11]:
TableDebug.get().debug("table0", table0)

DEBUG table0 Table{columns=[ColumnVector{rows=4, type=INT32, nullCount=Optional[0], offHeap=(ID: 4 7f2f7caf27d0)}], cudfTable=139838462723856, rows=4}
GPU COLUMN 0 - NC: 0 DATA: DeviceMemoryBufferView{address=0x7f2eaaa00000, length=16, id=-1} VAL: null
COLUMN 0 - INT32
0 1
1 2
2 3
3 4
GPU COLUMN 0 - NC: 0 DATA: DeviceMemoryBufferView{address=0x7f2eaaa00000, length=16, id=-1} VAL: null
COLUMN 0 - INT32
0 1
1 2
2 3
3 4


In [13]:
table0.getColumn(0).sum()

Scalar{type=INT32 value=10} (ID: 7 7f2eb6822f10)